In [ ]:
# @title Start Server (STEP 1)
from IPython.display import clear_output()
!pip install pypdf langchain langchain-community langchain-chroma langchain-ollama ollama chromadb PyPDF2 jupyterlab  django
!curl -L https://ollama.com/download/ollama-linux-amd64.tgz -o ollama-linux-amd64.tgz
!tar -C /usr -xzf ollama-linux-amd64.tgz
from subprocess import Popen
Popen("ollama serve".split())
clear_output()

In [ ]:
# @title Start Server (STEP 2)
!ollama pull llama3.2
!ollama pull nomic-embed-text
clear_output()

In [ ]:
# @title Start Server (STEP 3)
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
import time
from google.colab import files

persist_directory = 'data1'
localmodel = "llama3.1"
embeding = "nomic-embed-text"

def pdflrn():
  mainfile = files.upload()
  mf = list(mainfile)[0]
  filepath = mf


  loader = PyPDFLoader(filepath)
  data = loader.load()


  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
  all_splits = text_splitter.split_documents(data)


  vectorstore = Chroma.from_documents(all_splits, embedding=OllamaEmbeddings(model=embeding), persist_directory=persist_directory)
  return True

def pdfres(query):
  vectorstore1 = Chroma(embedding_function=OllamaEmbeddings(model=embeding), persist_directory=persist_directory)
  llm = OllamaLLM(base_url="http://localhost:11434", model=localmodel, verbose=True,
              callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
  retriever = vectorstore1.as_retriever()

  template = """ Answer the question based only on the following
  context: {context}

  User: {question}
  Chatbot:
  """
  prompt = PromptTemplate(input_variables=["context","question"], template=template,)
  #memory = ConversationBufferMemory(memory_key="history", return_messages=True, input_key="question")

  qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, verbose=False,
                                        chain_type_kwargs={
                                            #"verbose": True,
                                            "prompt": prompt,
                                            #"memory": memory,
                                        })
  res = qa_chain.invoke({"query": query})
  return res

In [ ]:
# @title Learn and Query
query = "" # @param {"type":"string"}
pdflrn()
clear_output()
re1 = pdfres(query)